# Подготовка окружения

## Установка пакетов и импорт зависимостей

In [2]:
!pip install datasets
!pip install pandas
!pip install flask
!pip install pyngrok
!pip install pytest

In [7]:
import sys
import importlib
import os
import subprocess
import requests
import json
from getpass import getpass
import logging

from typing import Tuple, List

from google.colab import drive
from joblib import dump, load

import torch

## Настройка окружения

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


## Клонирование исходных кодов проекта

In [11]:
def clone_pull_github_src(pull: bool = True):
    """
    Клонирует или обновляет репозиторий GitHub в локальный каталог для последующей работы.

    Parameters:
    pull (bool): Указывает, следует ли выполнять pull для существующего репозитория. Если True, выполняется git pull.
                 Если False, репозиторий клонируется в указанный каталог.

    Returns:
    Constants: Экземпляр класса Constants, содержащий константы проекта.
    """
    WORKSPACE_PATH = '/content/drive/MyDrive/docs/keepForever/mipt/nlp/hw_4sem/'
    WORKSPACE_TMP = WORKSPACE_PATH + '/tmp/'
    GIT_HUB_PROJECT_PATH = WORKSPACE_PATH + 'code/'

    token = getpass('Введите GitHub token: ')
    repo_url = 'https://github.com/km-mipt-nlp-gen/hw2.git'
    repo_url_with_token = repo_url.replace('https://', f'https://{token}@')

    os.chdir(GIT_HUB_PROJECT_PATH)

    if pull:
        !git pull origin main
    else:
        !git clone {repo_url_with_token} "$GIT_HUB_PROJECT_PATH"

    del token

    sys.path.append(f"{GIT_HUB_PROJECT_PATH}/web_app/src/")
    import constants_module
    importlib.reload(constants_module)
    from constants_module import Constants

    return Constants()

constants = clone_pull_github_src()
from constants_module import Constants
from chat_util_module import ChatUtil

Введите GitHub token: ··········
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 6 (delta 4), reused 6 (delta 4), pack-reused 0
Unpacking objects: 100% (6/6), 504 bytes | 7.00 KiB/s, done.
From https://github.com/km-mipt-nlp-gen/hw2
 * branch            main       -> FETCH_HEAD
   c905009..52bcc1d  main       -> origin/main
Updating c905009..52bcc1d
Fast-forward
 web_app/src/chat_controller_module.py | 39 ---------------------------------------
 web_app/src/chat_service_module.py    |  2 +-
 2 files changed, 1 insertion(+), 40 deletions(-)
DEVICE: cuda:0
Число процессов для использования: 12


## Обновление модулей

In [12]:
def reload_modules(constants: Constants) -> Tuple[Constants, ChatUtil]:
    """
    Перезагружает модули проекта для обновления изменений в коде без перезапуска среды выполнения.

    Parameters:
    constants (Constants): Экземпляр класса Constants, содержащий постоянные поля.

    Returns:
    Tuple[Constants, ChatUtil]: Кортеж, содержащий обновленный экземпляр класса Constants и экземпляр ChatUtil.
    """
    import sys
    sys.path.append(f"{constants.GIT_HUB_PROJECT_PATH}/web_app/src/")
    sys.path.append(f"{constants.GIT_HUB_PROJECT_PATH}/ml/src/train/")

    import importlib
    import chat_repository_module
    import chat_service_module
    import chat_controller_module
    import chat_util_module
    import constants_module
    import models_zoo_module
    import run_web_app_script

    importlib.reload(chat_repository_module)
    importlib.reload(chat_service_module)
    importlib.reload(chat_controller_module)
    importlib.reload(chat_util_module)
    importlib.reload(constants_module)
    importlib.reload(models_zoo_module)
    importlib.reload(run_web_app_script)

    from constants_module import Constants
    from chat_util_module import ChatUtil
    from chat_repository_module import ChatRepository
    from chat_service_module import ChatService
    from chat_controller_module import ChatController
    from run_web_app_script import run_web_app

    constants = Constants()

    return constants, ChatUtil(logging.DEBUG, constants), run_web_app

constants, chat_util, run_web_app = reload_modules(constants)
preprocessed_data = load(constants.PROCESSED_QA_PATH)

DEVICE: cuda:0
Число процессов для использования: 12


# Web-приложение

## Интеграционные тесты Web-приложения

In [13]:
def run_integration_tests(constants: Constants) -> None:
    """
    Запускает интеграционные тесты веб-приложения.

    Parameters:
    constants (Constants): Экземпляр класса Constants, содержащий постоянные поля проекта.

    Returns:
    None
    """
    os.chdir(constants.WEB_APP_TEST_PATH)

    os.environ['WEB_APP_SRC_PATH'] = constants.WEB_APP_SRC_PATH

    command = [
        "pytest", "-vv",
        constants.TEST_SCRIPT_PATH
    ]

    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    print(result.stdout)
    print(result.stderr)

run_integration_tests(constants)

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/drive/MyDrive/docs/keepForever/mipt/nlp/hw_4sem/code/web_app/test
plugins: anyio-3.7.1
collecting ... collected 3 items

test.py::test_that_expected_length_when_find_top_3_times[/gpt2-6] PASSED                     [ 33%]
test.py::test_that_expected_length_when_find_top[/gpt2-2] PASSED                             [ 66%]
test.py::test_that_clear_chat_when_find_top_then_clear[/gpt2] PASSED                         [100%]

======================================== 3 passed in 6.04s =========================================




## Старт Web-приложения

In [15]:
run_web_app()

··········


DEBUG:chat_util_module: * ngrok tunnel "https://b91e-34-126-65-49.ngrok-free.app" -> "http://127.0.0.1:5000/"


<Flask 'chat_controller_module'>

 * Serving Flask app 'chat_controller_module'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
!sudo fuser -k 4040/tcp # прервать процесс сервиса
# !sudo fuser -k 5000/tcp
# !ps aux | grep ngrok
# !ps aux | grep 4040

4040/tcp:            29352


### Запросы к серверу (через облачный туннель)

#### Одиночный вопрос и ответ

In [16]:
url = 'https://b91e-34-126-65-49.ngrok-free.app'

def process_queries(query_user_pairs: List[Tuple[str, str]], resource_url: str) -> None:
    """
    Реализация режима одиночного запроса и ответа.
    Обрабатывает серию запросов к API и выводит отформатированные ответы:
    - для пары пользователь - реплика формирует запрос;
    - далее отправляет запрос на 'top_cos_sim_bi_cr', 'top_l2_bi_cr', 'top_l2_psa_bi_cr', 'top_cr';
    - выводит ответы напротив каждого запроса;
    - очищает чат после каждого ответа.

    Parameters:
    query_user_pairs (List[Tuple[str, str]]): Список кортежей, содержащих пары пользователь-запрос.
    resource_url (str): Базовый URL ресурса, к которому будут отправляться запросы.

    Returns:
    None
    """
    endpoints = [
        'gpt2'
    ]
    clear_endpoint = '/clear'

    for index, (user, query) in enumerate(query_user_pairs, start=1):
        data = {
            "query": query,
            "user": user
        }

        for endpoint in endpoints:
            full_url = f'{resource_url}/{endpoint}'
            response = requests.post(full_url, json=data)

            formatted_response = json.dumps(response.json(), ensure_ascii=False, indent=2) if response.ok else response.text
            print(f'{index} "/{endpoint}" запрос:"{json.dumps(data, ensure_ascii=False)}" > ответ:"{formatted_response}"')

            requests.delete(f'{resource_url}{clear_endpoint}')

            if endpoint != endpoints[-1]:
                print('---')

        if index != len(query_user_pairs):
            print('=================================================================\n')

# вопрос - ответ (без сохранения контекста после ответа)
query_user_pairs = [
    ('Marge Simpson', 'Kids, you want to go Christmas shopping?'),
    ('Marge Simpson', 'Ready for Christmas, shopaholics?'),
    ('Ivan Ivanov', 'Ready for Christmas, shopaholics?'),
    ('Ivan Ivanov', 'Write a letter to Santa to Russia to Veliky Ustyug.'),
    ('Bart Simpson', 'Good one, Dad.'),
    ('Bart Simpson', 'Awesome, Dad.'),
    ('Igor Pokrishkin', 'Awesome, Dad.'),
    ('Igor Pokrishkin', 'Top-notch skill, Dad.')
]

process_queries(query_user_pairs, url)

DEBUG:chat_util_module:get_answer_gpt2_model - старт выполнения
DEBUG:chat_util_module:Query: Kids, you want to go Christmas shopping?
DEBUG:chat_util_module:User: Marge Simpson
DEBUG:chat_util_module:Обогащенный контекстом запрос: R_1: "Marge Simpson: Kids, you want to go Christmas shopping?"
DEBUG:chat_util_module:gpt модели ответ: It's a good thing Dad's got a big Santa Clause.
DEBUG:chat_util_module:Тело ответа gpt2: ['Marge Simpson: Kids, you want to go Christmas shopping?', "It's a good thing Dad's got a big Santa Clause."]
INFO:werkzeug:127.0.0.1 - - [11/Mar/2024 18:13:31] "POST /gpt2 HTTP/1.1" 200 -


1 "/gpt2" запрос:"{"query": "Kids, you want to go Christmas shopping?", "user": "Marge Simpson"}" > ответ:"{
  "response": [
    "Marge Simpson: Kids, you want to go Christmas shopping?",
    "It's a good thing Dad's got a big Santa Clause."
  ]
}"


INFO:werkzeug:127.0.0.1 - - [11/Mar/2024 18:13:32] "DELETE /clear HTTP/1.1" 200 -


DEBUG:chat_util_module:get_answer_gpt2_model - старт выполнения
DEBUG:chat_util_module:Query: Ready for Christmas, shopaholics?
DEBUG:chat_util_module:User: Marge Simpson
DEBUG:chat_util_module:Обогащенный контекстом запрос: R_1: "Marge Simpson: Ready for Christmas, shopaholics?"
DEBUG:chat_util_module:gpt модели ответ: (FOUGHT THE TRICK.
DEBUG:chat_util_module:Тело ответа gpt2: ['Marge Simpson: Ready for Christmas, shopaholics?', '(FOUGHT THE TRICK.']
INFO:werkzeug:127.0.0.1 - - [11/Mar/2024 18:13:34] "POST /gpt2 HTTP/1.1" 200 -


2 "/gpt2" запрос:"{"query": "Ready for Christmas, shopaholics?", "user": "Marge Simpson"}" > ответ:"{
  "response": [
    "Marge Simpson: Ready for Christmas, shopaholics?",
    "(FOUGHT THE TRICK."
  ]
}"


...

7 "/gpt2" запрос:"{"query": "Awesome, Dad.", "user": "Igor Pokrishkin"}" > ответ:"{
  "response": [
    "Igor Pokrishkin: Awesome, Dad.",
    "And your sister was killed by a snake."
  ]
}"


INFO:werkzeug:127.0.0.1 - - [11/Mar/2024 18:13:45] "DELETE /clear HTTP/1.1" 200 -


DEBUG:chat_util_module:get_answer_gpt2_model - старт выполнения
DEBUG:chat_util_module:Query: Top-notch skill, Dad.
DEBUG:chat_util_module:User: Igor Pokrishkin
DEBUG:chat_util_module:Обогащенный контекстом запрос: R_1: "Igor Pokrishkin: Top-notch skill, Dad."
DEBUG:chat_util_module:gpt модели ответ: I think you're a good one.
DEBUG:chat_util_module:Тело ответа gpt2: ['Igor Pokrishkin: Top-notch skill, Dad.', "I think you're a good one."]
INFO:werkzeug:127.0.0.1 - - [11/Mar/2024 18:13:46] "POST /gpt2 HTTP/1.1" 200 -


8 "/gpt2" запрос:"{"query": "Top-notch skill, Dad.", "user": "Igor Pokrishkin"}" > ответ:"{
  "response": [
    "Igor Pokrishkin: Top-notch skill, Dad.",
    "I think you're a good one."
  ]
}"


INFO:werkzeug:127.0.0.1 - - [11/Mar/2024 18:13:47] "DELETE /clear HTTP/1.1" 200 -


#### Диалог

In [19]:
def process_queries(query_user_pairs: List[Tuple[str, str]], resource_url: str) -> None:
    """
    Реализация режима диалога.
    Обрабатывает серию запросов к API и выводит состояние чата после каждого запроса:
    - для каждого из 'top_cos_sim_bi_cr', 'top_l2_bi_cr', 'top_l2_psa_bi_cr', 'top_cr' выполняет серию запросов;
    - запрос состввляется для пары пользователь - реплика;
    - после обработки всех запросов - переходит к отправке сообщений на следующий ресурс;
    - выводит текущего чата напротив каждого запроса;
    - очищает чат после завершения работы с ресурсом (после отправки всех запросов на текущий ресурс).

    Parameters:
    query_user_pairs (List[Tuple[str, str]]): Список кортежей, содержащих пары пользователь-запрос.
    resource_url (str): Базовый URL ресурса, к которому будут отправляться запросы.

    Returns:
    None
    """
    endpoints = [
        'gpt2'
    ]
    clear_endpoint = '/clear'

    for endpoint in endpoints:
        for index, (user, query) in enumerate(query_user_pairs, start=1):
            data = {
                "query": query,
                "user": user
            }


            full_url = f'{resource_url}/{endpoint}'
            response = requests.post(full_url, json=data)

            formatted_response = json.dumps(response.json(), ensure_ascii=False, indent=2) if response.ok else response.text
            print(f'{index} "/{endpoint}" запрос:"{json.dumps(data, ensure_ascii=False)}" > состояние чата:"{formatted_response}"')

            if index != len(query_user_pairs):
                print('---')

        requests.delete(f'{resource_url}{clear_endpoint}')
        if endpoint != endpoints[-1]:
            print('=================================================================\n')

# вопрос - состояние чата (с сохранением контекста)
query_user_pairs = [
    ('Postman', 'Your Cristmas Tree.'),
    ('Postman', 'From North Pole.')
]

resource_url =  url
process_queries(query_user_pairs, resource_url)

DEBUG:chat_util_module:get_answer_gpt2_model - старт выполнения
DEBUG:chat_util_module:Query: Your Cristmas Tree.
DEBUG:chat_util_module:User: Postman
DEBUG:chat_util_module:Обогащенный контекстом запрос: R_1: "Postman: Your Cristmas Tree."
DEBUG:chat_util_module:gpt модели ответ: I am, Mrs. Marge Simpson...
DEBUG:chat_util_module:Тело ответа gpt2: ['Postman: Your Cristmas Tree.', 'I am, Mrs. Marge Simpson...']
INFO:werkzeug:127.0.0.1 - - [11/Mar/2024 18:16:28] "POST /gpt2 HTTP/1.1" 200 -


1 "/gpt2" запрос:"{"query": "Your Cristmas Tree.", "user": "Postman"}" > состояние чата:"{
  "response": [
    "Postman: Your Cristmas Tree.",
    "I am, Mrs. Marge Simpson..."
  ]
}"
---


DEBUG:chat_util_module:get_answer_gpt2_model - старт выполнения
DEBUG:chat_util_module:Query: From North Pole.
DEBUG:chat_util_module:User: Postman
DEBUG:chat_util_module:Обогащенный контекстом запрос: R_3: "Postman: Your Cristmas Tree."; R_2: "I am, Mrs. Marge Simpson..."; R_1: "Postman: From North Pole."
DEBUG:chat_util_module:gpt модели ответ: From North Pole.
DEBUG:chat_util_module:Тело ответа gpt2: ['Postman: Your Cristmas Tree.', 'I am, Mrs. Marge Simpson...', 'Postman: From North Pole.', 'From North Pole.']
INFO:werkzeug:127.0.0.1 - - [11/Mar/2024 18:16:29] "POST /gpt2 HTTP/1.1" 200 -


2 "/gpt2" запрос:"{"query": "From North Pole.", "user": "Postman"}" > состояние чата:"{
  "response": [
    "Postman: Your Cristmas Tree.",
    "I am, Mrs. Marge Simpson...",
    "Postman: From North Pole.",
    "From North Pole."
  ]
}"


INFO:werkzeug:127.0.0.1 - - [11/Mar/2024 18:16:29] "DELETE /clear HTTP/1.1" 200 -
